In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
import numpy as np
import pandas as pd
from torch.utils.data import Dataset
import math
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import time

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
class AmericanExpressProfileTimeSeriesDataset(Dataset):
    def __init__(self, dataset_file, transformation=False, pandas_transformation=False):
        self.dataset = pd.read_pickle(dataset_file)
        self.transformation = transformation
        self.pandas_transformation = pandas_transformation
        if self.pandas_transformation: self.dataset = self.pandas_transformation(self.dataset)

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        row = self.dataset.iloc[[idx]]
        label = row["target"].values
        data = row.drop(['customer_ID', 'target'], axis=1)
        data = data.values[0].tolist()
        data = np.array(data, dtype=np.float32)
        data = torch.tensor(data, dtype=torch.float32, requires_grad=True)
        label = torch.tensor(label, dtype=torch.float32)
        label = label.to(device)
        data = data.to(device)
        if self.transformation: data = self.transformation(data)
        return data.T, label

In [4]:
full_dataset = AmericanExpressProfileTimeSeriesDataset("transformed_dataset")

train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, test_size])

train_dataloader = DataLoader(train_dataset, batch_size=64)
test_dataloader = DataLoader(test_dataset, batch_size=64)

In [7]:
class ConvolutionalNetwork(nn.Module):

    def __init__(self):
        super(ConvolutionalNetwork, self).__init__()

        self.conv = nn.Sequential(
            nn.Conv1d(197, 256, 2, stride=1),
            nn.ReLU(),
            nn.Conv1d(256, 512, 2, stride=1),
            nn.ReLU(),
            nn.MaxPool1d(2, stride=1),
            nn.Conv1d(512, 1024, 2, stride=1),
            nn.ReLU(),
            nn.Conv1d(1024, 1024, 2, stride=1),
            nn.ReLU(),
            nn.MaxPool1d(2, stride=1),
            nn.Conv1d(1024, 1024, 2, stride=1),
            nn.ReLU(),
            nn.Conv1d(1024, 1024, 2, stride=1),
            nn.ReLU(),
            nn.MaxPool1d(5, stride=1),
            nn.Flatten(),
            nn.Linear(1024, 256),
            nn.ReLU(),
            nn.Linear(256, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.conv(x)
        return x

model = ConvolutionalNetwork()
model = model.to(device)

In [33]:
class LSTM1(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, seq_length):
        super(LSTM1, self).__init__()
        self.num_layers = num_layers #number of layers
        self.input_size = input_size #input size
        self.hidden_size = hidden_size #hidden state
        self.seq_length = seq_length #sequence length

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True) #lstm
        self.fc_1 =  nn.Linear(200, 128)
        self.fc = nn.Linear(128, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
    
    def forward(self,x):
        h_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        output, (hn, cn) = self.lstm(x, (h_0, c_0))
        hidden = torch.cat((hn[-2,:,:], hn[-1,:,:]), dim = 1)     
        out = self.relu(hidden)
        out = self.fc_1(out)
        out = self.relu(out)
        out = self.fc(out)
        out = self.sigmoid(out)
        return out

model = LSTM1(197, 100, 5, 3)
model = model.to(device)

In [9]:
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            pred = torch.round(pred)
            correct += (pred == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    return test_loss

In [10]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        X = X.to(device)
        y = y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [11]:
models = {}

In [34]:
batch_size = 64
epochs = 1000

loss_fn = nn.BCELoss()
optimizer = torch.optim.AdamW(model.parameters())
#optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
#scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    models[t] = model
    test_loss = test_loop(test_dataloader, model, loss_fn)
    #scheduler.step(test_loss)
print("Done!")

Epoch 1
-------------------------------
loss: 0.722039  [    0/367130]
loss: 0.241055  [ 6400/367130]
loss: 0.386259  [12800/367130]
loss: 0.203367  [19200/367130]
loss: 0.213593  [25600/367130]
loss: 0.204404  [32000/367130]
loss: 0.267252  [38400/367130]
loss: 0.194315  [44800/367130]
loss: 0.187997  [51200/367130]
loss: 0.246286  [57600/367130]
loss: 0.312783  [64000/367130]
loss: 0.400745  [70400/367130]
loss: 0.229945  [76800/367130]
loss: 0.313509  [83200/367130]
loss: 0.269525  [89600/367130]
loss: 0.218962  [96000/367130]
loss: 0.223600  [102400/367130]
loss: 0.256982  [108800/367130]
loss: 0.213133  [115200/367130]
loss: 0.166121  [121600/367130]
loss: 0.159059  [128000/367130]
loss: 0.284331  [134400/367130]
loss: 0.252891  [140800/367130]
loss: 0.222183  [147200/367130]
loss: 0.346799  [153600/367130]
loss: 0.273046  [160000/367130]
loss: 0.129720  [166400/367130]
loss: 0.207607  [172800/367130]
loss: 0.331847  [179200/367130]
loss: 0.268282  [185600/367130]
loss: 0.308793  

In [ ]:
torch.save(models[4], "./model")
#model = torch.load("./model")
#model.eval()

In [ ]:
model_ = models[2]

In [ ]:
full_validation_dataset = AmericanExpressProfileTimeSeriesDataset("transformed_test_dataset")
validation_dataloader = DataLoader(full_validation_dataset, batch_size=64)

In [ ]:
def validation_loop(dataloader, model, out_file):

    with torch.no_grad():
        data_file = open(f"{out_file}.csv", "w")
        data_file.write("customer_ID,prediction\n")
        for X, customer_ID in dataloader:
            pred = model(X)
            pred = pred.tolist()
            for idx in range(len(customer_ID)):
                data_file.write(customer_ID[idx] + "," + str(pred[idx])+"\n")
        data_file.close()

In [ ]:
validation_loop(validation_dataloader, model, "./test_data/test_labels")